### Training on Admission data

In [1]:
import numpy as np
import pandas as pd

admissions = pd.read_csv('binary.csv')

In [2]:
admissions.head(10)

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
5,1,760,3.00,2
6,1,560,2.98,1
7,0,400,3.08,2
8,1,540,3.39,3
9,0,700,3.92,2


### data processing and standardizing

In [3]:
# Make dummy variables for rank

data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

In [4]:
data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,380,3.61,0,0,1,0
1,1,660,3.67,0,0,1,0
2,1,800,4.00,1,0,0,0
3,1,640,3.19,0,0,0,1
4,0,520,2.93,0,0,0,1


In [5]:
data.loc[:,'gre'][:5]

0    380
1    660
2    800
3    640
4    520
Name: gre, dtype: int64

In [6]:
# standarize features, since gpa and gre score are higher in range 
# (x-μ) / σ

for field in ['gre','gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:, field] = (data[field]-mean)/std

In [7]:
data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
0,0,-1.798011,0.578348,0,0,1,0
1,1,0.625884,0.736008,0,0,1,0
2,1,1.837832,1.603135,1,0,0,0
3,1,0.452749,-0.525269,0,0,0,1
4,0,-0.586063,-1.208461,0,0,0,1


In [8]:
np.random.seed(42)

In [9]:
data.index

RangeIndex(start=0, stop=400, step=1)

In [10]:
# split random 10% data for testing

sample = np.random.choice(data.index, size = int(len(data) * 0.9), replace=False)

In [11]:
len(sample)

360

In [12]:
sample[:10]

array([209, 280,  33, 210,  93,  84, 329,  94, 266, 126], dtype=int64)

note: here sample returns the index within the data points, so later we can use these sample index for training data and 
    remaining index location data points to the test data

In [13]:
data, test_data = data.ix[sample], data.drop(sample)

C:\Users\DELL\Anaconda3\envs\nirmalenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [14]:
# training data
data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,0,-0.066657,0.289305,0,1,0,0
280,0,0.625884,1.445476,0,1,0,0
33,1,1.837832,1.603135,0,0,1,0
210,0,1.318426,-0.131120,0,0,0,1
93,0,-0.066657,-1.208461,0,1,0,0


In [15]:
# test data
test_data.head()

,admit,gre,gpa,rank_1,rank_2,rank_3,rank_4
20,0,-0.759199,-0.577822,0,0,1,0
21,1,0.625884,0.630901,0,1,0,0
48,0,-1.278605,-2.390908,0,0,0,1
50,0,0.452749,1.235263,0,0,1,0
54,0,0.625884,-0.131120,0,0,1,0


In [16]:
# split into features and target 

features, target = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [17]:
features.head()

,gre,gpa,rank_1,rank_2,rank_3,rank_4
209,-0.066657,0.289305,0,1,0,0
280,0.625884,1.445476,0,1,0,0
33,1.837832,1.603135,0,0,1,0
210,1.318426,-0.131120,0,0,0,1
93,-0.066657,-1.208461,0,1,0,0


In [18]:
target.head()

209    0
280    0
33     1
210    0
93     0
Name: admit, dtype: int64

In [19]:
features.shape

(360, 6)

In [20]:
target.shape

(360,)

In [21]:
features.values

array([[-0.06665712,  0.28930534,  0.        ,  1.        ,  0.        ,
         0.        ],
       [ 0.62588442,  1.44547565,  0.        ,  1.        ,  0.        ,
         0.        ],
       [ 1.83783211,  1.60313523,  0.        ,  0.        ,  1.        ,
         0.        ],
       ...,
       [-0.41292789, -0.57782239,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.62588442,  1.60313523,  1.        ,  0.        ,  0.        ,
         0.        ],
       [-0.41292789, -0.28877981,  1.        ,  0.        ,  0.        ,
         0.        ]])

In [22]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 2000
learnrate = 0.6

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, target):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = sigmoid(np.dot(weights,x))

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate the error term
        error_term = error * (output * (1-output))

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w +=  error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += (learnrate * del_w) / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - target) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Train loss:  0.26247839036458864
Train loss:  0.19964393498481797
Train loss:  0.19747964768672152
Train loss:  0.19711407642258863
Train loss:  0.1970145428076338
Train loss:  0.19698153970950913
Train loss:  0.19696947849039506
Train loss:  0.19696483445463397
Train loss:  0.19696299207911608
Train loss:  0.19696224793246683
Prediction accuracy: 0.725
